<a href="https://colab.research.google.com/github/Liztomania/NLP_Kurs_2024/blob/main/test_visualisierung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Visualisierung

Dataset importieren
Dataset an unsere Anfrage angepasst und für den Test auf 10 Zeilen reduziert (von ursprünglich 200)

In [17]:
%pip install -q pysolr


import pandas as pd
import pysolr

solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'
solr = pysolr.Solr(solr_url, always_commit=True, timeout=30)

q = {
    'q': 'plainpagefulltext:"China~ Erdbeben~"~35 AND publication_year:[1920 TO 1921]',
    'rows': 10
}

response = solr.search(**q)

# Überführen der Ergebnisse in ein Pandas DataFrame
df_1 = pd.DataFrame(response.docs)

# DataFrame anzeigen
df_1

KeyboardInterrupt: 

Impresso NER und NEL API (alles Folgende kopiert aus https://ieg-dhr.github.io/NLP-Course4Humanities_2024/modules/module_6.html Notebook Named Entity Recognition Impresso API)  

In [11]:
import requests

def get_linked_entities(text, coarse_only=False):
    """
    Calls the external API to get named entity recognition (NER) results.
    """
    url = "https://impresso-annotation.epfl.ch/api/ner/"
    payload = {"data": text}
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            data = response.json()
            data["text"] = text
            # remove fine-grained and components
            if coarse_only:
                for ne in data["nes"]:
                    data["nes"] = [ne for ne in data["nes"] if not "." in ne["type"]]
            return data
        else:
            print(f"Request failed with status code {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
import requests
import pandas as pd
import json

def get_linked_entities(text, coarse_only=False):
    """
    Calls the external API to get named entity recognition (NER) results.
    """
    url = "https://impresso-annotation.epfl.ch/api/ner/"
    payload = {"data": text}
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            data = response.json()
            data["text"] = text
            # remove fine-grained and components
            if coarse_only:
                data["nes"] = [ne for ne in data["nes"] if not "." in ne["type"]]
            return data
        else:
            print(f"Request failed with status code {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Assuming 'extracted_article_clean' is the column name in your DataFrame
# geändert zu `place of distribution' aus unserer Tabelle
def process_dataframe(df):
    results = []
    for index, row in df.iterrows():
        text = row['place_of_distribution']
        api_result = get_linked_entities(text, coarse_only=True)
        results.append(api_result)
    df['NER_results'] = results
    return df

# hier code angepasst: articles_df wird zu einfach nur df
df_1 = process_dataframe(df_1)
df_1.head()

??? Frage: warum sind es hier nur noch 4 bzw. 5 Zeilen? Im Original auch nur 5, aber unklar, woher das kommt

# Extracting Locations from NER Output

In [ ]:
#extract locations

import pandas as pd

def extract_locations(df):
    places = []
    for index, row in df.iterrows():
        ner_results = row['NER_results']
        if ner_results and 'nes' in ner_results:
            # Use 'surface' instead of 'text' to get the entity text
            location_names = [ne['surface'] for ne in ner_results['nes'] if ne.get('type') == 'loc']
            places.append(', '.join(location_names))  # Join multiple locations
        else:
            places.append('')  # Handle cases where NER_results is None or nes key is missing
    df['places'] = places
    return df

df_1 = extract_locations(df_1)
print(df_1[['place_of_distribution', 'places']].head())